In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
!wget https://f000.backblazeb2.com/file/malaya-model/v39/summarization/base.pb
!wget https://f000.backblazeb2.com/file/malaya-model/v39/summarization/small.pb

--2020-11-01 22:13:38--  https://f000.backblazeb2.com/file/malaya-model/v39/summarization/base.pb
Resolving f000.backblazeb2.com (f000.backblazeb2.com)... 104.153.233.177
Connecting to f000.backblazeb2.com (f000.backblazeb2.com)|104.153.233.177|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 831717841 (793M) [application/octet-stream]
Saving to: ‘base.pb’

base.pb             100%[===================>] 793.19M  14.1MB/s    in 76s     

2020-11-01 22:14:55 (10.5 MB/s) - ‘base.pb’ saved [831717841/831717841]

--2020-11-01 22:14:56--  https://f000.backblazeb2.com/file/malaya-model/v39/summarization/small.pb
Resolving f000.backblazeb2.com (f000.backblazeb2.com)... 104.153.233.177
Connecting to f000.backblazeb2.com (f000.backblazeb2.com)|104.153.233.177|:443... connected.
HTTP request sent, awaiting response... 200 
Length: 378870799 (361M) [application/octet-stream]
Saving to: ‘small.pb’

small.pb            100%[===================>] 361.32M  12.2MB/s    in 38s    

In [7]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.set_random_seed(0)

In [8]:
pbs = glob('*.pb')
pbs

['small.pb', 'base.pb']

In [4]:
import tensorflow_text
import tf_sentencepiece

In [10]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_constants(ignore_errors=true)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

for pb in pbs:
    input_graph_def = tf.GraphDef()
    with tf.gfile.FastGFile(pb, 'rb') as f:
        input_graph_def.ParseFromString(f.read())
        
    print(pb)
    
    transformed_graph_def = TransformGraph(input_graph_def, 
                                           ['Placeholder', 'Placeholder_2'],
                                           ['greedy', 'beam', 'nucleus'], transforms)
    
    with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
        f.write(transformed_graph_def.SerializeToString())

small.pb
base.pb


In [11]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [13]:
import malaya

model = malaya.summarization.abstractive.transformer(model = 'small')

downloading frozen /home/husein/Malaya/summarize/transformer/small model


362MB [00:29, 12.4MB/s]                          


downloading frozen /home/husein/Malaya/summarize/transformer/small vocab


132%|██████████| 1.00/0.76 [00:01<00:00, 1.04s/MB]




In [14]:
string = 'KUALA LUMPUR: Presiden Perancis Emmanuel Macron tidak menampakkan beliau seorang sosok yang bertamadun, selar Tun Dr Mahathir Mohamad menerusi kemas kini terbaharu di blognya. Bekas Perdana Menteri itu mendakwa, pemerintah tertinggi Perancis itu bersikap primitif kerana menuduh orang Islam terlibat dalam pembunuhan guru yang menghina Islam, malah menegaskan tindakan membunuh bukan ajaran Islam. Jelas Dr Mahathir, sejarah membuktikan bahawa orang Perancis pernah membunuh jutaan manusia, yang ramai mangsanya terdiri dari orang Islam.'

In [15]:
model.summarize([string])

['Presiden Perancis tidak menampakkan figur seorang lelaki yang bertanggungjawab']

In [22]:
e = model._tokenizer.encode(f'ringkasan: {string}')
e = e + [1]

In [17]:
g = load_graph('small.pb.quantized')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/greedy:0')
test_sess = tf.InteractiveSession(graph = g)

In [24]:
l = test_sess.run(logits, feed_dict = {x: [e]})

In [26]:
model._tokenizer.decode(l[0].tolist())

'Presiden Perancis Emmanuel Macron tidak menampakkan sosok yang menyimpang, selar Tun M'

In [27]:
quantized = glob('*.pb.quantized')
quantized

['small.pb.quantized', 'base.pb.quantized']

In [ ]:
!rm *.pb*

In [ ]:
# converter = tf.compat.v1.lite.TFLiteConverter.from_frozen_graph(
#     graph_def_file='test.pb',
#     input_arrays=['Placeholder', 'Placeholder_1'],
#     input_shapes={'Placeholder' : [None, 512], 'Placeholder_1': [None, 512]},
#     output_arrays=['logits'],
# )
# # converter.allow_custom_ops=True

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True
# tflite_model = converter.convert()

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-float16.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, 
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# tflite_model = converter.convert()

# with open('tiny-bert-sentiment-hybrid.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# interpreter = tf.lite.Interpreter(model_path='tiny-bert-sentiment-hybrid.tflite')
# interpreter.allocate_tensors()